In [13]:
import os, sys
import numpy as np
import qcportal as ptl
import pandas as pd


In [14]:
def load_QM9(input_file=None, output_file=None):
    """Load the QM9 datset sourced from qcportal
    
    Parameters
    ----------
    input_file : str, optional, default=None
        The locally stored hdf5 file of the QM9 dataset to load. 
        If not provided, the data will be fetched from qcarchive.
    output_file : str, optional, default=None
        The name defining where to save the dataset locally.
        If input_file and output_file are both None, the dataset will be deleted upon exit.

    Returns
    --------
    qcportal.collections.dataset.Dataset, 
        The qm9 dataset
    """
    qcp_client = ptl.FractalClient()

    # to get QM9 from qcportal, we need to define which collection and QM9
    # we will first check to see if it exists 
    qcportal_data = {'collection': 'Dataset', 'dataset': 'QM9'}

    try: 
        qm9 = qcp_client.get_collection(qcportal_data['collection'], qcportal_data['dataset'])
    except:
        print(f"Dataset {qcportal_data['dataset']} is not available in collection {qcportal_data['collection']}.")
    
    if input_file is None:
        # if we don't define output_file below, the data we download
        # will be deleted upon exit.
        if not output_file is None:
            qm9.download()
        else:
            qm9.download(local_path=output_file)

        """
        # note this is failing due to issues with distutils in qcportal
        # 

        if not output_file is None:
            #need some error checking here.  probably want to raise errors in the real code
            assert output_file.endswith('.hdf5')
            qm9.to_file(path=output_file, encoding='hdf5')
        """  
    else:
        # again, better error checking is probably required
        # but this is probably good enough
        assert input_file.endswith('.hdf5')
        qm9.set_view(input_file)

    
    return qm9
    

In [15]:
def parse_data(qm9):
    
    """Parse the QM9 datset into a dictionaries

    Note: this function takes a while to run, as retrieving the records
    appears to be a bit slow given the dataset size.
    
    Parameters
    ----------
    qm9 : qcportal.collections.dataset.Dataset
        The Dataset to parse
   
    Returns
    --------
    tuple, (list, dict, dict) 
        The first index is a list of molecule names
        The second index is a dictionary of molecule geometry information, keyed by name.
        the third index is a dictionary of molecule QM properties, keyed by name.
    """
    
    molecules = qm9.get_molecules()

    # just going to restrict to this for demo purposes now
    # this could easily be an argument
    records  = qm9.get_records(method='b3lyp')

    # the order of molecules and records in the dataframes provided by qcportal 
    # is not guaranteed. Use dicts keyed by the named for each property.
    
    records_dict = {} 
    molecules_dict = {}

    names = []
    
    for i in range(molecules.shape[0]):
        name = molecules.iloc[i].name
        molecules_dict[name] = molecules.iloc[i][0]
        names.append(name)
        
    for i in range(len(records)):
        rec = records.iloc[i].record
        name = records.index[i]
        records_dict[name] = rec
        
    return (names, molecules_dict, records_dict)

In [16]:
qm9 = load_QM9(input_file='/Users/cri/Documents/Projects-msk/qcarchive/qm9.hdf5')

In [17]:
%time names, molecules, records = parse_data(qm9)

CPU times: user 1min 1s, sys: 14.6 s, total: 1min 16s
Wall time: 11min 16s


In [18]:
7/11

0.6363636363636364

Let us just look at the arbitrary molecule from the names list to explore the structure of the information from qcportal. 

Just call an individual molecule will default to visualizing it. 

In [6]:
molecules[names[200]]

NGLWidget()

In [ ]:
print(molecules[names[200]])

For each molecule, we can see the info stored via the `dict` function which, as the function name suggests, returns a dictionary of the molecule properties/info.

In [7]:
molecules[names[200]].dict()

{'schema_name': 'qcschema_molecule',
 'schema_version': 2,
 'validated': True,
 'symbols': array(['C', 'C', 'N', 'C', 'C', 'N', 'C', 'C', 'H', 'H', 'H', 'H', 'H',
        'H', 'H', 'H'], dtype='<U1'),
 'geometry': array([[-1.39051210e-01,  2.83318818e+00, -1.65827600e-02],
        [-5.01442800e-02, -6.25411000e-03,  5.31336900e-02],
        [ 3.64649400e-02, -1.19749836e+00, -2.18241027e+00],
        [ 1.16266390e-01, -3.72038431e+00, -2.16145846e+00],
        [ 2.13221080e-01, -5.05322009e+00, -4.66981902e+00],
        [ 1.17849420e-01, -5.18739469e+00, -9.15737600e-02],
        [ 3.14795800e-02, -3.98373004e+00,  2.11532670e+00],
        [-5.59533500e-02, -1.36624272e+00,  2.30630329e+00],
        [ 1.52032710e+00,  3.57197760e+00, -1.00609064e+00],
        [-1.79899626e+00,  3.46631919e+00, -1.07595343e+00],
        [-2.04921830e-01,  3.64663564e+00,  1.87950305e+00],
        [ 1.91393720e+00, -6.22452692e+00, -4.79036663e+00],
        [-1.40453398e+00, -6.32768411e+00, -4.86377329e

In [10]:
names[0]

'dsgdb9nsd_087380'

We can easily access any of the quantities listed in the dict, e.g., geometry.  Some of the properties have multiple access points, e.g, could also called `molecules[names[200]].geometry`

In [11]:
molecules[names[0]].dict()['geometry']

array([[-0.04041566,  2.74344493, -0.65039891],
       [ 0.08658344, -0.0432305 , -0.0912989 ],
       [ 2.51601522, -1.21345819, -0.20646512],
       [ 1.29307705, -1.14870993,  2.17623057],
       [ 2.3617901 , -0.04071865,  4.66922479],
       [ 1.61007472,  2.60608   ,  5.51963152],
       [ 1.12950728, -2.30096348,  6.12986142],
       [ 0.41850848, -3.50320787,  3.57589577],
       [-0.42497824, -5.49861056,  2.92809407],
       [ 1.56663697,  3.74232639,  0.16847553],
       [-0.01539865,  3.045327  , -2.69750579],
       [-1.79044758,  3.56241374,  0.08895625],
       [-1.40628846, -1.18569908, -0.94301317],
       [ 4.42202371, -0.22821917,  4.65510567],
       [-0.44152455,  2.86942755,  5.43861731],
       [ 2.20744337,  2.92552075,  7.47485868],
       [ 2.48903828,  4.07080792,  4.35653387],
       [ 2.3775782 , -3.44021712,  7.31562907],
       [-0.53583381, -1.7731344 ,  7.24008159]])

Similarly, we can see the QM related info stored for each record.  

In [12]:
records[names[0]].dict()

{'id': '4980271',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {},
 'extras': {'qcvars': {'CURRENT DIPOLE X': -0.4094730292125962,
   'CURRENT DIPOLE Y': 3.365837001975174,
   'CURRENT DIPOLE Z': 1.4725107550328682,
   'CURRENT ENERGY': -422.83362082526696,
   'CURRENT REFERENCE ENERGY': -422.83362082526696,
   'DFT FUNCTIONAL TOTAL ENERGY': -422.833620825267,
   'DFT TOTAL ENERGY': -422.833620825267,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -47.61689880437887,
   'NUCLEAR REPULSION ENERGY': 455.7265718596327,
   'ONE-ELECTRON ENERGY': -1476.8942384679592,
   'PCM POLARIZATION ENERGY': 0.0,
   'SCF DIPOLE X': -0.4094730292125962,
   'SCF DIPOLE Y': 3.365837001975174,
   'SCF DIPOLE Z': 1.4725107550328682,
   'SCF ITERATION ENERGY': -422.83362082526696,
   'SCF ITERATIONS': 13.0,
   'SCF TOTAL ENERGY': -422.83362082526696,
   'TWO-ELECTRON ENERGY': 645.9509445874384,
   'XC GRID RADIAL POINTS': 100.0,
   'XC GRID SPHERICAL POINTS': 

The qcvars entry under extras houses a summary of the key QM data. Like geometry, extras can also be accessed either through the dict or its own entry point to get the qcvars info. Important note, if you check the units of the dataset `qm9.units`, it will report kcal/mol, however, the energy values given here are hatree.

In [ ]:
records[names[200]].extras['qcvars']

Also interesting, the entire output from the run can be accessed. 

In [ ]:
print(records[names[200]].get_stdout())